In [1]:
#importing necessary libraries

import cv2
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans
import glob
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

In [3]:
# reading all our required datasets
path = "/home/subarna/Documents/vr/MT2019514_MT2019523_MT2019079/Part2/CIFAR-10-images-master/train"
filenames = glob.glob(path + "/*")
df_train = []
descriptor = []

for filename in filenames:
    clas_nams = glob.glob(filename + "/*")
    for clas_nam in clas_nams:
        try:
            imageID = filename[filename.rfind("/") + 1:]
            img = cv2.imread(clas_nam)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            extractor = cv2.xfeatures2d.SIFT_create()
            keypoints, descriptors = extractor.detectAndCompute(img, None)
            descriptor.extend(descriptors)
            df_train.append((imageID,descriptors))
        except TypeError as e:
            print(e)

'NoneType' object is not iterable
'NoneType' object is not iterable
'NoneType' object is not iterable
'NoneType' object is not iterable
'NoneType' object is not iterable
'NoneType' object is not iterable
'NoneType' object is not iterable
'NoneType' object is not iterable
'NoneType' object is not iterable
'NoneType' object is not iterable
'NoneType' object is not iterable
'NoneType' object is not iterable
'NoneType' object is not iterable
'NoneType' object is not iterable
'NoneType' object is not iterable
'NoneType' object is not iterable
'NoneType' object is not iterable
'NoneType' object is not iterable
'NoneType' object is not iterable
'NoneType' object is not iterable
'NoneType' object is not iterable
'NoneType' object is not iterable
'NoneType' object is not iterable
'NoneType' object is not iterable
'NoneType' object is not iterable
'NoneType' object is not iterable
'NoneType' object is not iterable
'NoneType' object is not iterable
'NoneType' object is not iterable
'NoneType' obj

In [4]:
# reading all our required datasets
path = "/home/subarna/Documents/vr/MT2019514_MT2019523_MT2019079/Part2/CIFAR-10-images-master/test"
filenames = glob.glob(path + "/*")
df_test = []
descriptor_test = []
for filename in filenames:
    clas_nams = glob.glob(filename + "/*")
    for clas_nam in clas_nams:
        try:
            imageID = filename[filename.rfind("/") + 1:]
            img = cv2.imread(clas_nam)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            extractor = cv2.xfeatures2d.SIFT_create()
            keypoints, descriptors = extractor.detectAndCompute(img, None)
            descriptor_test.extend(descriptors)
            df_test.append((imageID,descriptors))
        except TypeError as e:
            print(e)

'NoneType' object is not iterable
'NoneType' object is not iterable
'NoneType' object is not iterable
'NoneType' object is not iterable
'NoneType' object is not iterable
'NoneType' object is not iterable
'NoneType' object is not iterable
'NoneType' object is not iterable
'NoneType' object is not iterable
'NoneType' object is not iterable
'NoneType' object is not iterable
'NoneType' object is not iterable
'NoneType' object is not iterable
'NoneType' object is not iterable
'NoneType' object is not iterable
'NoneType' object is not iterable


In [8]:
des_train = np.array(descriptor)

In [10]:
des_train.shape

(663279, 128)

In [29]:
#extracting our clusters and there centroids
kmeans = KMeans(n_clusters = 40, random_state = 10)
kmeans.fit(des_train)
cluster = kmeans.cluster_centers_

In [31]:
kmeans

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=40, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=10, tol=0.0001, verbose=0)

In [15]:
df_tr = pd.DataFrame(df_train)
df_te = pd.DataFrame(df_test)

In [16]:
l = {'dog' : 0, 'automobile' : 1, 'bird' : 2, 'airplane' : 3, 'ship' : 4, 'truck' : 5, 'frog' : 6, 'horse' : 7, 'deer' : 8, 'cat' : 9}
df_tr[0] = [l[item] for item in df_tr[0]]
df_te[0] = [l[item] for item in df_te[0]]

In [17]:
x_test = df_tr[0].to_numpy()
y_test = df_te[0].to_numpy()

In [53]:
x = np.array(x_test)

In [60]:
np.reshape(df_train[0][1][0], (1,128)).shape

(1, 128)

In [32]:
from scipy.cluster.vq import vq
train_features = np.zeros((len(df_train), 40), "float32")
for i in range(0,len(df_train)):
    words, distance = vq(df_train[i][1],kmeans.cluster_centers_)
    for w in words:
        train_features[i][w] += 1

test_features = np.zeros((len(df_test), 40), "float32")
for i in range(0,len(df_test)):
    words, distance = vq(df_test[i][1],kmeans.cluster_centers_)
    for w in words:
        test_features[i][w] += 1

In [33]:
### preproceesing train and test dataset 
#from sklearn.preprocessing import StandardScaler
#Perform Tf-Idf vectorization
nbr_occurences = np.sum( (train_features > 0) * 1, axis = 0)
# Calculating the number of occurrences
idf = np.array(np.log((1.0*len(train_features)+1) / (1.0*nbr_occurences + 1)), 'float32')
# Giving weight to one that occurs more frequently
# Scaling the features
stdSlr = StandardScaler().fit(train_features)
im_features_train = stdSlr.transform(train_features)

nbr_occurences = np.sum( (test_features > 0) * 1, axis = 0)
idf = np.array(np.log((1.0*len(test_features)+1) / (1.0*nbr_occurences + 1)), 'float32')
stdSlr = StandardScaler().fit(test_features)
im_features_test = stdSlr.transform(test_features)

In [20]:
model = KNeighborsClassifier(n_neighbors=7,n_jobs=-1)
model.fit(im_features_train, x_test)
acc = model.score(im_features_test, y_test)
print("Accuracy: {:.2f}%".format(acc * 100))

Accuracy: 19.00%


In [34]:
model = KNeighborsClassifier(n_neighbors=9,n_jobs=-1)
model.fit(im_features_train, x_test)
acc = model.score(im_features_test, y_test)
print("Accuracy: {:.2f}%".format(acc * 100))

Accuracy: 19.45%


[('building', array([[ 0.,  0., 27., ..., 19.,  9.,  5.],
         [ 0.,  0.,  0., ..., 17.,  3.,  0.],
         [ 2.,  0.,  0., ...,  2., 27., 27.],
         ...,
         [ 4.,  5., 69., ...,  6.,  2.,  2.],
         [48., 95.,  3., ...,  0., 19., 24.],
         [ 0.,  0.,  1., ..., 11., 70., 23.]], dtype=float32)),

array([[ 8.260876 ,  8.000003 , 14.804345 , ..., 16.239134 ,  7.6739125,
        11.413044 ],
       [35.813564 , 17.271187 , 10.661015 , ..., 10.000004 ,  9.898304 ,
        14.559323 ],
       [37.456142 , 31.052639 , 18.421051 , ..., 19.859648 ,  8.087721 ,
        18.789474 ],
       ...,
       [11.434788 ,  7.608696 , 32.565216 , ..., 14.782609 , 27.130432 ,
        25.913044 ],
       [ 5.511116 ,  9.044447 , 30.088886 , ...,  5.666665 ,  6.3555565,
         6.3555555],
       [41.431377 , 68.352936 , 46.07843  , ...,  8.803923 , 12.9803915,
        19.784313 ]], dtype=float32)

cluster.shape
(50, 128)

In [35]:
df_train_vlad = []
for i in range(0,len(df_train)):
    su = np.zeros((1,128))
    for j in range(0,len(df_train[i][1])):
        ind = kmeans.predict([df_train[i][1][j]])
        x = df_train[i][1][j] - cluster[ind]
        su += x
    df_train_vlad.append(su)

In [85]:
df_train[0][1].shape

(16, 128)

In [96]:
model = SVC(kernel='linear')
model.fit(im_features_train, x_test)
acc = model.score(im_features_test, y_test)
print("Accuracy: {:.2f}%".format(acc * 100))

Accuracy: 25.79%


In [88]:
vlad_features=[]
for i in range(len(df_train)):
    vlad_vector = np.zeros((40,128), "float32")
    words, distance = vq(df_train[i][1],kmeans.cluster_centers_)
    j=0
    for w in words:
        vlad_vector[w] = np.add(vlad_vector[w], np.subtract(df_train[i][1][j],kmeans.cluster_centers_[w]))
        j = j+1
    norm = np.linalg.norm(vlad_vector)
    vlad_vector= np.divide(vlad_vector,norm)
    vlad_vector= vlad_vector.flatten()
    vlad_features.append(vlad_vector)

In [91]:
vlad_features_test=[]
for i in range(len(df_test)):
    vlad_vector = np.zeros((40,128), "float32")
    words, distance = vq(df_test[i][1],kmeans.cluster_centers_)
    j=0
    for w in words:
        vlad_vector[w] = np.add(vlad_vector[w], np.subtract(df_test[i][1][j],kmeans.cluster_centers_[w]))
        j = j+1
    norm = np.linalg.norm(vlad_vector)
    vlad_vector= np.divide(vlad_vector,norm)
    vlad_vector= vlad_vector.flatten()
    vlad_features_test.append(vlad_vector)

In [95]:
from sklearn.svm import SVC
model = SVC(kernel='linear')
model.fit(vlad_features, x_test)
acc = model.score(vlad_features_test, y_test)
print("Accuracy: {:.2f}%".format(acc * 100))

Accuracy: 34.96%
